In [49]:
import itertools
import torch
from torch.utils.data import DataLoader
from model import MatchOutcomeTransformer, FixedPrefixDataset, collate_fn

In [58]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(torch.backends.mps.is_available())  # Should print True
print(torch.backends.mps.is_built())     

True
True


In [59]:
point_match_tensor = torch.load("/Users/bowmannovey/Desktop/Transformer_Tennis/tennis_transformer_model/Saved_Dataframes/point_match_tensors.pt")
labels = torch.load("/Users/bowmannovey/Desktop/Transformer_Tennis/tennis_transformer_model/Saved_Dataframes/labels.pt")

/var/folders/pc/kr77rnc562z4p366739tyz6w0000gn/T/ipykernel_17172/4060776224.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  point_match_tensor = torch.load("/Users/bowma

In [60]:
dataset = FixedPrefixDataset(point_match_tensor, labels, min_prefix=5, prefixes_per_match=1, max_prefix=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
input_dim = point_match_tensor[0].shape[1]


In [61]:
len(dataset)

6530

In [62]:
param_grid = {
    'lr': [1e-4],
    'dropout': [0.1, 0.3],
    'd_model': [128, 256],
    'num_layers': [2, 6],
    'nhead' : [2, 4] }
param_combinations = list(itertools.product(*param_grid.values()))

In [63]:
results = []

In [64]:
for lr, dropout, d_model, num_layers, nhead in param_combinations:
    print(f"Testing: d_model={d_model}, nhead={nhead}, num_layers={num_layers}, dropout={dropout}, lr={lr}")
    model = MatchOutcomeTransformer(
        input_dim=25,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()

    # One epoch of training (for demonstration; use more for real sweep)
    for epoch in range(3):
        model.train()
        running_loss = 0.0
        running_correct = 0
        total = 0
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            preds = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * batch_x.size(0)
            preds_label = (preds > 0.5).float()
            running_correct += (preds_label == batch_y).sum().item()
            total += batch_y.size(0)
        epoch_loss = running_loss / total
        epoch_acc = running_correct / total
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")


    # Evaluate (dummy example, replace with real validation)
    results.append({
        'params': (d_model, nhead, num_layers, dropout, lr),
        'loss': epoch_loss,
        'accuracy': epoch_acc
    })

Testing: d_model=128, nhead=2, num_layers=2, dropout=0.1, lr=0.0001


KeyboardInterrupt: 

In [32]:
print(results)

[{'params': (128, 2, 2, 0.1, 0.0001), 'val_loss': 0.7966236472129822}, {'params': (128, 4, 2, 0.1, 0.0001), 'val_loss': 0.7248304486274719}, {'params': (128, 2, 4, 0.1, 0.0001), 'val_loss': 0.7170495986938477}, {'params': (128, 4, 4, 0.1, 0.0001), 'val_loss': 0.7193442583084106}, {'params': (128, 2, 6, 0.1, 0.0001), 'val_loss': 0.6615761518478394}, {'params': (128, 4, 6, 0.1, 0.0001), 'val_loss': 0.7081653475761414}, {'params': (256, 2, 2, 0.1, 0.0001), 'val_loss': 0.6939610242843628}, {'params': (256, 4, 2, 0.1, 0.0001), 'val_loss': 0.7334239482879639}, {'params': (256, 2, 4, 0.1, 0.0001), 'val_loss': 0.7963665723800659}, {'params': (256, 4, 4, 0.1, 0.0001), 'val_loss': 0.6594635248184204}, {'params': (256, 2, 6, 0.1, 0.0001), 'val_loss': 0.6741803884506226}, {'params': (256, 4, 6, 0.1, 0.0001), 'val_loss': 0.7958598136901855}, {'params': (128, 2, 2, 0.3, 0.0001), 'val_loss': 0.6615808606147766}, {'params': (128, 4, 2, 0.3, 0.0001), 'val_loss': 0.7011159658432007}, {'params': (128, 2,